# Week 6: The Poisson Process

**Objective:** Understand the Poisson process as a model for counting random events in continuous time, learn its fundamental properties, and simulate it to verify its theoretical characteristics.

## Step 1: Build Intuition

Imagine you're working at a call center. Phone calls seem to arrive at random, but you know that on average, you get about 10 calls per hour. The Poisson process is the perfect tool to model this scenario.

It's a **counting process**. It counts the number of events (e.g., phone calls, emails, customers arriving, radioactive decays) that have occurred up to a certain point in time. 

The key intuitive ideas are:
1.  Events happen at a constant average **rate**.
2.  The timing of one event tells you nothing about when the next one will occur (the process is **memoryless**).
3.  Events don't happen at the exact same time.

## Step 2: Understand the Core Idea

The Poisson process, denoted \(N(t)\), counts the number of events up to time \(t\). It's defined by a few simple but powerful properties:

1.  **Starts at Zero:** The count begins at time 0, so \(N(0) = 0\).
2.  **Independent Increments:** The number of events in one time interval is independent of the number of events in any *non-overlapping* time interval. (The number of calls between 9 AM and 10 AM doesn't affect the number of calls between 2 PM and 3 PM).
3.  **Stationary Increments:** The probability distribution of the number of events depends only on the *length* of the time interval, not its location. (The probability of getting 5 calls in any 1-hour window is the same, whether it's 9-10 AM or 2-3 PM).
4.  **Rate of Events:** For a very small time interval \(h\), the probability of one event occurring is approximately \(\lambda h\), where \(\lambda\) is the constant rate of events. The probability of more than one event is essentially zero.

## Step 3: Learn the Definitions and Formulas

**Definition: Poisson Process**
A counting process \({N(t), t \ge 0}\) is a Poisson process with rate \(\lambda > 0\) if it satisfies the core properties listed above.

--- 

**Key Result 1: Distribution of the Number of Events**
The number of events \(N(t)\) that occur in any interval of length \(t\) follows a **Poisson distribution** with mean \(\lambda t\).
$$ P(N(t) = k) = \frac{e^{-\lambda t} (\lambda t)^k}{k!}, \quad k = 0, 1, 2, ... $$

--- 

**Key Result 2: Distribution of Inter-arrival Times**
Let \(T_i\) be the time between the \((i-1)\)-th and the \(i\)-th event. These are called the **inter-arrival times**. For a Poisson process, the inter-arrival times \(T_1, T_2, ...\) are independent and identically distributed random variables, each following an **Exponential distribution** with rate \(\lambda\).
$$ f_{T_i}(t) = \lambda e^{-\lambda t}, \quad t \ge 0 $$
The mean inter-arrival time is \(E[T_i] = 1/\lambda\). This makes perfect sense: if calls arrive at a rate of 10 per hour, the average time between calls is 1/10th of an hour (6 minutes).

## Step 4: Apply and Practice

The relationship between the Poisson process and the Exponential distribution gives us a beautiful way to simulate it. We can generate the random times *between* events and then add them up to find the time *at which* each event occurs.

**Scenario:** Let's simulate a Poisson process for a call center where calls arrive at an average rate of \(\lambda = 5\) calls per hour, over a period of 2 hours.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

plt.style.use('seaborn-v0_8-whitegrid')

### Part A: Simulating a Poisson Process Path

We will simulate the process by generating exponential inter-arrival times.

In [ ]:
def simulate_poisson_process(rate, time_period):
    """
    Simulates a single path of a Poisson process.
    
    Args:
        rate (float): The arrival rate (lambda).
        time_period (float): The total time to simulate for.
        
    Returns:
        tuple: A tuple containing (event_times, event_counts).
    """
    event_times = [0]
    current_time = 0
    
    while current_time < time_period:
        # Generate the next inter-arrival time from an Exponential distribution
        # The scale parameter for numpy's exponential is 1/lambda.
        inter_arrival_time = np.random.exponential(scale=1.0/rate)
        
        current_time += inter_arrival_time
        
        if current_time < time_period:
            event_times.append(current_time)
            
    # The count of events at each event time
    event_counts = np.arange(len(event_times))
    
    return event_times, event_counts

# --- Simulation Parameters ---
LAMBDA = 5      # Rate: 5 calls per hour
T_PERIOD = 2    # Period: 2 hours

# Run one simulation
times, counts = simulate_poisson_process(LAMBDA, T_PERIOD)

# Plot the Poisson process path
plt.figure(figsize=(12, 6))
plt.step(times, counts, where='post')
plt.title(f'A Single Realization of a Poisson Process ($\lambda={LAMBDA}$)')
plt.xlabel('Time (Hours)')
plt.ylabel('Number of Events N(t)')
plt.xlim(0, T_PERIOD)
plt.grid(True)
plt.show()

print(f"Total events in {T_PERIOD} hours: {len(counts)-1}")

The plot shows a typical path of a counting process. It's a step function that jumps up by 1 at each random event time.

### Part B: Verifying the Distribution of N(t)

The theory says that the number of events in our 2-hour window, \(N(2)\), should follow a Poisson distribution with mean \(\lambda t = 5 \times 2 = 10\).

Let's test this by running many simulations and plotting a histogram of the total number of events.

In [ ]:
N_SIMS = 10000
total_events_in_sims = []

for _ in range(N_SIMS):
    times, counts = simulate_poisson_process(LAMBDA, T_PERIOD)
    total_events_in_sims.append(len(counts) - 1)

# Plot the histogram of the results
plt.figure(figsize=(12, 6))
max_events = max(total_events_in_sims)
bins = np.arange(0, max_events + 2) - 0.5
plt.hist(total_events_in_sims, bins=bins, density=True, alpha=0.7, label='Simulation Results')

# Overlay the theoretical Poisson PMF
mu = LAMBDA * T_PERIOD
x = np.arange(0, max_events + 1)
pmf = poisson.pmf(x, mu)
plt.plot(x, pmf, 'ro-', label=f'Theoretical Poisson PMF ($\mu={mu}$)')

plt.title(f'Distribution of N({T_PERIOD}) over {N_SIMS} Simulations')
plt.xlabel('Total Number of Events in 2 Hours')
plt.ylabel('Probability')
plt.legend()
plt.show()

print(f"Average number of events from simulation: {np.mean(total_events_in_sims):.4f}")
print(f"Theoretical average number of events: {mu}")

**Interpretation:**

The results are a perfect match! The histogram of event counts from our simulation aligns beautifully with the theoretical Poisson probability mass function. The simulated average number of events is also extremely close to the theoretical mean of 10. This confirms our understanding and simulation method.

## Summary & Next Steps

In this notebook, we've explored the Poisson process:
1.  It's a **continuous-time counting process** for random, independent events occurring at a constant average rate \(\lambda\).
2.  The number of events in an interval of length \(t\) follows a **Poisson(\(\lambda t\)) distribution**.
3.  The time *between* consecutive events follows an **Exponential(\(\lambda\)) distribution**.
4.  We can simulate the process by generating these exponential inter-arrival times.

The Poisson process is a cornerstone of stochastic modeling, especially in queuing theory and reliability engineering. 

In **Week 7**, we will return to discrete-time processes but with a focus on time series data, introducing **Linear Systems Theory and Autoregressive (AR) Models**.